# Demonstration of the `PartitionExt` class
This notebook provides example code to demonstrate the usage of the `PartitionExt` class, its methods, and related functions defined in the `abacus_extension.py` module. A SageMath notebook kernel is required to use built-in SageMath functionality, such as that provided by the module `sage.combinat.partition`. Check the `README` for installation instructions and documentation.

## Setup/Imports
First, we import the `PartitionExt` class from `abacus_extension.py` along with helper functions for constructing class instances from different data formats.
We also import a collecton of functions which perform unit tests from the `tests.py` module.

In [1]:
from abacus_extension import PartitionExt, from_G_core_and_quotient, from_G_charges_and_quotient, from_G_abacus, invert_zero_one
from tests import test_from_G_core_and_quotient, test_from_G_abacus, test_special_core, test_special_quotient

## `PartitionExt` methods

The `PartitionExt` class defined in `abacus_extension.py` extends the `Partition` class in SageMath with the following methods:
    
* `G_colour_tableau(r, b=-1)` - Returns tableau of `self` with cells coloured by $(i,j) \mapsto i + bj \ (\mathrm{mod}\ r)$.
* `G_colour_count(r, b=-1)` - Counts the number of cells in `self` of each colour under the $(r,b)$-colouring.
* `is_G_core(r, b=-1)` - Checks whether `self` is a G-core with respect to the $(r,b)$-action
* `G_core(r, b=-1)` - Returns the G-core partition of `self` with respect to the $(r,b)$-action
* `G_quotient(r, b=-1)` - Returns the G-quotient of `self` with respect to the $(r,b)$-action, an $r$-tuple of partitions
* `G_abacus(r, b=-1)` - Returns an $r$-tuple of path sequences {1:N, 0:E} corresponding to `self` and the $(r,b)$-action
* `G_charges(r, b=-1)` - Returns the charge coordinates of `self` with respect to the $(r,b)$-action, an $r$-tuple of integers
    
In the default case that only one argument `r` is passed, the action is of type $(r,-1) = (r,r-1)$ which is special linear and 
yields the same result as the classical `core(r)` and `quotient(r)` methods.

In [2]:
p = PartitionExt(Partition([5,4]))
r, b = (3, 1)

In [3]:
print(f"p = {p}, {type(p)}")
print(f"Is `p` a ({r},{b})-core? {p.is_G_core(r,b)}")
print(f"({r},{b})-core-quotient decomposition: core = {p.G_core(r,b)}, quotient = {p.G_quotient(r,b)}")
print(f"({r},{b})-abacus: {p.G_abacus(r,b)}")
print(f"({r},{b})-charge coordinates: {p.G_charges(r,b)}")
print(f"({r},{b})-colour tableau: {p.G_colour_tableau(r,b)}")
print(f"({r},{b})-colour counts: {p.G_colour_count(r,b)}")

p = [5, 4], <class 'abacus_extension.PartitionExt'>
Is `p` a (3,1)-core? False
(3,1)-core-quotient decomposition: core = [2, 1], quotient = ([1, 1], [], [])
(3,1)-abacus: [deque([0, 1, 1]), deque([0]), deque([0, 0, 0])]
(3,1)-charge coordinates: [-2, 1, 1]
(3,1)-colour tableau: [[0, 1, 2, 0, 1], [1, 2, 0, 1]]
(3,1)-colour counts: [3, 4, 2]


## Unit testing

Given a group action defined by parameters $(r,b)$ and a partition $p$, we verify that we can reconstruct $p$ after decomposing it into its $(r,b)$-core and quotient pair, or into its $(r,b)$-abacus representation.
Specifically we test the following conditions:
``` python
p == from_G_core_and_quotient(p.G_core(r,b), p.G_quotient(r,b), r, b)
p == from_G_abacus(p.G_abacus(r,b), r, b)
```
We perform these tests for all partitions of size less than `max_p_size`, 
and for all nondegenerate actions of a cyclic group $G$ of size $r$ less than `max_r`.<br>
That is, for all partitions $p$ such that $0 \le \mathrm{size}(p) < \text{max_p_size}$ and all $0 < r < \text{max_r}$, $b \in \mathbb{Z}_{r}^{\times}$.

In [4]:
test_from_G_core_and_quotient(max_r=9, max_p_size=20) and test_from_G_abacus(max_r=9, max_p_size=20)

Reconstructing all partitions of size less than 20 from their G-core-quotient decomposition
For nondegenerate actions of a cyclic group G of size r up to 9
Test passed for r = 1
Test passed for r = 2
Test passed for r = 3
Test passed for r = 4
Test passed for r = 5
Test passed for r = 6
Test passed for r = 7
Test passed for r = 8
Reconstructing all partitions of size less than 20 from their minimal G-abacus
For nondegenerate actions of a cyclic group G of size r up to 9
Test passed for r = 1
Test passed for r = 2
Test passed for r = 3
Test passed for r = 4
Test passed for r = 5
Test passed for r = 6
Test passed for r = 7
Test passed for r = 8


True

Next, we test that our generalised implementation of `PartitionExt.G_core(r,b)` coincides with the existing `Partition.core(r)` method when $b \equiv -1 (\mathrm{mod}\ r)$. e.g. when $b = r - 1 \text{ or } -1$.

This should reflect the mathematical fact that the classical $r$-core of a partition (obtained by removing border strips of length $r$) coincides with the generalized $(r,b)$-core in this special case.

We perform analogous checks comparing `PartitionExt.G_quotient(r, b=-1)` with `Partition.quotient(r)`.
We perform these tests for all partitions of size less than `max_p_size` and values of the modulus $r$ less than `max_r`, whose defaults are $20$ and $15$, respectively.

Note that due to differences in conventions effectively swapping xy-coordinates for cell colouring (vs. content), the order of partitions in the $(r,r-1)$-quotient differs from the classical $r$-quotient by a reflection of indices, i.e. the order of the $r$-tuple of partitions is reversed. 
This can be accounted for by setting the optional `label_swap_xy` keyword argument to `True` when calling the `PartitionExt.G_quotient` method. See the docstrings for `PartitionExt.G_quotient` and `test_special_quotient` for further details.

In [5]:
test_special_core() and test_special_quotient()

True

## Future Work
### Experimental evidence for research questions:
- Test that, for any fixed core, increasing quotient size does not decrease partition size (or decreasing quotient size does not increase partition size)
- Compare coloured box count coordinates with charge coordinates (should be linear change of variables)

### Expand unit testing
* Implement `test_from_G_charges_and_quotient` - Tests the bijection between partitions and G-charges-quotient decomposition
- Testing that partitions constructed from a valid core and quotient pair or abacus recover their initial data correctly
- Expand tests with known small/hand-calculated examples in the unit testing suite?
- Improve/expand documentation where unclear (Feedback welcome!)

### Implementation details
- Consider whether or not to pass r,b as (optional) class attributes so that group action is fixed (if useful to users)
- Other user suggestions for features. Please feel free to get in touch.